In [ ]:
import sys, time
import numpy as np
import arkouda as ak

In [ ]:
ak.connect(server="localhost", port=5555)

In [ ]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [ ]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [ ]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

In [ ]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

In [ ]:
print(nfDF)

In [ ]:
print(ak.info(ak.AllSymbols))

In [ ]:
%time u,c = nfDF['srcIP'].unique(return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = nfDF['dstIP'].unique(return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = nfDF['srcPort'].unique(return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = nfDF['dstPort'].unique(return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
print((nfDF['srcPort'] == 8080).sum())
print((nfDF['dstPort'] == 8080).sum())

In [ ]:
s = u[0:65] # pick off the first 100 dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
s = u[0:65].to_ndarray() # pick off the first 100 dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
#ak.shutdown()
#ak.disconnect()